In [3]:
import os
project_path = "c:\\Users\\gufer\\OneDrive\\Documentos\\FIAP\\Fase_03\\"
os.chdir(project_path)

In [5]:
import pandas as pd
from classes.deck.constants import color_combinations, color_combinations_abbreviated

df_cc = pd.DataFrame.from_dict(color_combinations, orient='index')
df_cca = pd.DataFrame.from_dict(color_combinations_abbreviated, orient='index')
df_colors = pd.concat([df_cc, df_cca], axis=0)
df_colors

,W,U,B,R,G
Monowhite,1,0,0,0,0
Monoblue,0,1,0,0,0
Monoblack,0,0,1,0,0
Monored,0,0,0,1,0
Monogreen,0,0,0,0,1
...,...,...,...,...,...
UGB,0,1,1,0,1
BUG,0,1,1,0,1
BGU,0,1,1,0,1
GBU,0,1,1,0,1


In [17]:
from typing import Dict, Union

def get_deck_colors(deck_name: str, 
                    df_colors: pd.DataFrame) -> Union[Dict[str, int], str]:
    """
    Identifica as cores de um deck de Magic: The Gathering com base no nome do deck.

    Args:
        deck_name (str): O nome do deck, que pode incluir combinações de cores abreviadas (ex. "RW") ou nomes completos (ex. "Selesnya").
        df_colors (pd.DataFrame): DataFrame contendo combinações de cores e seus valores binarizados.

    Returns:
        Union[Dict[str, int], str]: Um dicionário com as cores e seus valores binarizados (1 para presente, 0 para ausente) 
                                    ou uma mensagem de erro se a combinação de cores não for encontrada.
    """
    # Extrair letras maiúsculas do nome do deck para abreviações
    deck_letters = ''.join([char for char in deck_name if char.isupper()])
    deck_letters_sorted = ''.join(sorted(deck_letters))

    # Tentar localizar as cores no DataFrame concatenado usando as letras
    if deck_letters_sorted in df_colors.index:
        return df_colors.loc[deck_letters_sorted].to_dict()

    # Se não encontrar pelas letras, verificar os nomes completos
    for full_name in df_colors.index:
        if full_name.lower() in deck_name.lower():
            return df_colors.loc[full_name].to_dict()

    # Caso não encontre nenhuma correspondência
    return "Combinação de cores não encontrada para o deck"

In [18]:
# Testando a determinação de cores a partir do nome do deck
from deck import Deck

deck = Deck()
deck_name = "Selesnya Rabbits"
deck_colors = deck.determine_deck_colors_from_name(deck_name, df_colors)

print(f"As cores para o deck '{deck_name}' são: {deck_colors}")

As cores para o deck 'Selesnya Rabbits' são: {'W': 1, 'U': 0, 'B': 0, 'R': 0, 'G': 1}


In [21]:
# Carregue o nome do deck e as cartas a partir de um arquivo txt
deck.load_deck_from_txt("deck\\deck_samples\\sample_01.txt")

In [22]:
# Determine as cores a partir do nome do deck
deck.determine_deck_colors_from_name(deck.deck_name, df_colors)

{'W': 0, 'U': 0, 'B': 0, 'R': 1, 'G': 1}

In [23]:
from mtgsdk import Card

def is_tapped_land(card: Card) -> bool:
    """
    Determines if a land card enters the battlefield tapped.

    Args:
        card (Card): An instance of the Card class from mtgsdk.

    Returns:
        bool: True if the land enters tapped, False otherwise.
    """
    # Verifica se o card é um terreno (Land)
    if 'Land' not in card.type:
        return False
    
    # Verifica se o texto do card indica que ele entra em jogo virado
    if card.text and ('tapped' in card.text.lower() or 'enters the battlefield tapped' in card.text.lower()):
        return True
    
    return False

In [24]:
import random

#  Função para buscar e testar uma amostra aleatória de terrenos
def test_tapped_lands_sample(sample_size=10):
    # Buscar todos os terrenos (Land)
    lands = Card.where(type='Land').where(set='LTR').all()
    
    # Filtrar uma amostra aleatória
    sample_lands = random.sample(lands, min(sample_size, len(lands)))

    # Testar se os terrenos entram virados
    tapped_lands = [card.name for card in sample_lands if is_tapped_land(card)]
    non_tapped_lands = [card.name for card in sample_lands if not is_tapped_land(card)]

    return tapped_lands, non_tapped_lands

# Executar o teste com uma amostra de 10 terrenos
tapped_lands, non_tapped_lands = test_tapped_lands_sample(10)

print("Terrenos que entram virados:", tapped_lands)
print("Terrenos que não entram virados:", non_tapped_lands)

Terrenos que entram virados: ['Minas Tirith', 'Rivendell', 'Mines of Moria', 'The Shire']
Terrenos que não entram virados: ['Mountain', 'Mount Doom', 'Forest', 'Island', 'Plains', 'Mount Doom']


In [27]:
from player import Player

# Crie um jogador
player = Player(name="John", deck=deck)

# O jogador compra sua mão inicial
player.draw_initial_hand()
print(player.hand)  # Verifique a mão inicial

# Realize um mulligan
player.mulligan()
print(player.hand)  # Verifique a mão após o mulligan

ValueError: The deck provided is not valid.